In [117]:
import pandas as pd
import numpy as np
import os

from keras import utils
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D, MaxPooling2D, Conv2D
from keras import regularizers
from keras.layers import Embedding, Bidirectional,LSTM,TimeDistributed

In [100]:
df = pd.read_csv('NonPromoterSequence.txt', sep = '>', )
df.dropna(subset=['Unnamed: 0'], how='all', inplace=True)
df.reset_index(inplace = True)
df.drop(['EP 1 (+) mt:CoI_1; range -400 to -100.', 'index'], axis = 1, inplace=True)
df.rename(columns={'Unnamed: 0': "sequence"}, inplace = True)
df['label'] = 0
display(df)
display(df.shape)

,sequence,label
0,TAATTACATTATTTTTTTATTTACGAATTTGTTATTCCGCTTTTAT...,0
1,ATTTTTACAAGAACAAGACATTTAACTTTAACTTTATCTTTAGCTT...,0
2,AGAGATAGGTGGGTCTGTAACACTCGAATCAAAAACAATATTAAGA...,0
3,TATGTATATAGAGATAGGCGTTGCCAATAACTTTTGCGTTTTTTGC...,0
4,AGAAATAATAGCTAGAGCAAAAAACAGCTTAGAACGGCTGATGCTC...,0
...,...,...
11295,TGGTAAAAAATTGTACACCTAACTAGTGCCTTCATGTATACCACCA...,0
11296,AGTGCAACTGGAGCCGTGCCGTGACCCACAGAGATCGCCCACTCGA...,0
11297,GCATGGATTTCATATTATCTTAATCGACTTGCTTTTATAAAATAGG...,0
11298,GTGACCAGGTTTTGCTCTAATGCGAAGTACGGATTGGGTAGAGATA...,0


(11300, 2)

In [101]:
df2 = pd.read_csv('PromoterSequence.txt', sep = '>', )
df2.dropna(subset=['Unnamed: 0'], how='all', inplace=True)
df2.reset_index(inplace = True)
df2.drop(['EP 1 (+) mt:CoI_1; range -100 to 200.', 'index'], axis = 1, inplace=True)
df2.rename(columns={'Unnamed: 0': "sequence"}, inplace = True)
df2['label'] = 1

display(df2)
display(df2.shape)

,sequence,label
0,TTAATTTGTCCTTATTTGATTAAGAAGAATAAATCTTATATATAGA...,1
1,ATAGCTCAAATTGCTTTATTAGTATTAGAATCAGCTGTAGCTATAA...,1
2,AAGCTTCCCTTTAATGTGCTCCTTGTGAATACAGCATTACAATGCC...,1
3,TATGTAGAATCTGTACAAGTATCTGTGTTTGGACAATGGCATGTGT...,1
4,ACATATTACTGCATACAGGTCTCAAATTATAAAATGACACTCGTGG...,1
...,...,...
11295,CGACAAAGTTTGATCCATGTGCATTCTTGGCGCCTTATCGATAGCT...,1
11296,CATATCTACATCTCGCTTGCTCCTTCCCTTTCGCTGCGTGTGTGTG...,1
11297,ATACCGCGGAAGCGCAAAAGTACCAGAATTTCCCTGGTATCGCGCT...,1
11298,ATTATTCCGAATTCTTTTATCAGATTTAAATATGGGAAACACTTTA...,1


(11300, 2)

In [102]:
df = pd.concat([df, df2], axis = 0 )
df.shape

(22600, 2)

In [103]:
for seq in df['sequence']:
    if 'N' in seq:
        display(df.loc[df['sequence'] == seq])

,sequence,label
1822,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNGAATTC...,0


In [104]:
df.drop([1822], inplace = True)

In [105]:
for seq in df['sequence']:
    if 'N' in seq:
        display(df.loc[df['sequence'] == seq])

In [106]:
sequence = list(df.loc[:, 'sequence'])
encoded_list = []

In [107]:
def encode_seq(s):
    Encode = {'A':[1,0,0,0],'T':[0,1,0,0],'C':[0,0,1,0],'G':[0,0,0,1]}
    return [Encode[x] for x in s]

for i in sequence:
    x = encode_seq(i)
    encoded_list.append(x)

X = np.array(encoded_list)
X.shape

(22598, 301, 4)

In [112]:
X = X.reshape(X.shape[0],301, 4, 1)
X.shape

(22598, 301, 4, 1)

In [113]:
y = df['label']
y.shape

(22598,)

In [114]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)

In [115]:
y_train = utils.to_categorical(y_train)
y_test = utils.to_categorical(y_test)

In [123]:
model = Sequential()

model.add(TimeDistributed(Conv2D(filters = 27,       
                kernel_size = (3,3),    
                activation = 'relu',    
              input_shape = (301, 4,1))))
model.add(TimeDistributed(MaxPooling2D(pool_size= (2,2))))
model.add(TimeDistributed((Dropout(0.2))))

In [124]:
model.add(TimeDistributed(Flatten()))

model.add(Bidirectional(LSTM(64,return_sequences=True,input_shape = (X_train[0],301))))
model.add(LSTM(32))
#model.add(Dense(128, activation = 'relu', kernel_regularizer = regularizers.l2(0.001)))
#model.add(Dense(32, activation = 'relu', kernel_regularizer = regularizers.l2(0.001)))
model.add(Dense(2, activation = 'sigmoid'))

In [125]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [126]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [127]:
history = model.fit(X_train,
                    y_train, 
                    batch_size=256,
                    validation_data=(X_test, y_test),
                    epochs=10)

Epoch 1/10


ValueError: in user code:

    /Users/noah/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /Users/noah/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/noah/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/noah/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/noah/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/noah/opt/anaconda3/lib/python3.8/site-pack

In [84]:
X_train.shape

(16948, 301, 4, 1)

In [98]:
np.expand_dims(X_train,axis=0)

array([[[[[0],
          [0],
          [0],
          [1]],

         [[0],
          [1],
          [0],
          [0]],

         [[0],
          [1],
          [0],
          [0]],

         ...,

         [[0],
          [1],
          [0],
          [0]],

         [[0],
          [1],
          [0],
          [0]],

         [[0],
          [1],
          [0],
          [0]]],


        [[[0],
          [0],
          [1],
          [0]],

         [[0],
          [0],
          [0],
          [1]],

         [[0],
          [0],
          [1],
          [0]],

         ...,

         [[1],
          [0],
          [0],
          [0]],

         [[0],
          [0],
          [0],
          [1]],

         [[0],
          [1],
          [0],
          [0]]],


        [[[0],
          [0],
          [1],
          [0]],

         [[0],
          [0],
          [1],
          [0]],

         [[0],
          [1],
          [0],
          [0]],

         ...,

         [[0],
      

In [89]:
vocab_size = 4
embedding_dim = 64
max_length = 301

In [94]:
model = Sequential()
model.add(Embedding(vocab_size,embedding_dim,input_length=max_length))
model.add(Bidirectional(LSTM(64,return_sequences=True,input_shape = (X_train[0],301))))
model.add(LSTM(32))
model.add(Dense(2,activation = 'softmax'))

In [95]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [96]:
model.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 301, 64)           256       
_________________________________________________________________
bidirectional_17 (Bidirectio (None, 301, 128)          66048     
_________________________________________________________________
lstm_35 (LSTM)               (None, 301, 32)           20608     
_________________________________________________________________
dense_20 (Dense)             (None, 301, 2)            66        
Total params: 86,978
Trainable params: 86,978
Non-trainable params: 0
_________________________________________________________________


In [97]:
history = model.fit(X_train,
                    y_train, 
                    batch_size=256,
                    validation_data=(X_test, y_test),
                    epochs=10)

Epoch 1/10


ValueError: in user code:

    /Users/noah/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /Users/noah/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/noah/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/noah/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/noah/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/noah/opt/anaconda3/lib/python3.8/site-pack